In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, make_scorer
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)

# This folder has the final model
# It is a gradient boosting regressor using vector quintization

In [2]:
# Import dataset
df = pd.read_csv("train.csv")

In [3]:
# Define sections
lot = [
    "LotFrontage",
    "LotArea"
]

major = [
    "OverallQual",
    "OverallCond",
    "YearBuilt",
    "YearRemodAdd",
]

basement = [
    "BsmtFinSF1",
    "BsmtUnfSF",
    "TotalBsmtSF"
]

floors = [
    "1stFlrSF",
    "2ndFlrSF",
    "GrLivArea"
]

rooms = [
    "BedroomAbvGr",
    "KitchenAbvGr",
    "TotRmsAbvGrd",
    "Fireplaces",
]

bathrooms = [
    "BsmtFullBath",
    "FullBath",
    "HalfBath",
]

garage = [
    "GarageYrBlt",
    "GarageCars",
]
    
outdoors = [
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "ScreenPorch",
    "PoolArea",
    "MasVnrArea"
]

year = [
    "GarageYrBlt",
    "YearBuilt",
    "YearRemodAdd",
]

therest = np.concatenate([lot, major, garage, year])


In [4]:
# Make a pipeline to prepare data 
from sklearn.base import BaseEstimator, TransformerMixin
class FillNanValues(BaseEstimator, TransformerMixin):
    """
    Gets a DataFrame describing houses and fill the NaNs with meaningful replacements
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.LotFrontageMedian = X.LotFrontage.median()
        self.MasVnrAreaMedian = X.MasVnrArea.median()
        return self
    def transform(self, X):
        X.LotFrontage.fillna(self.LotFrontageMedian, inplace=True)
        X.MasVnrArea.fillna(self.MasVnrAreaMedian, inplace=True)
        X.MiscFeature.fillna("NoShed", inplace=True)
        X.Fence.fillna('NoFence', inplace=True)
        X.BsmtQual.fillna("NoBasement", inplace=True)
        X.BsmtCond.fillna("NoBasement", inplace=True)
        X.BsmtExposure.fillna("NoBasement", inplace=True)
        X.BsmtFinType1.fillna("NoBasement", inplace=True)
        X.BsmtFinType2.fillna("NoBasement", inplace=True)
        X.BsmtFinType2.fillna("NoBasement", inplace=True)
        X.MasVnrType.fillna("None", inplace=True)
        X.Electrical.fillna("SBrkr", inplace=True)
        X.FireplaceQu.fillna('NoFP', inplace=True)
        X.GarageType.fillna('Attchd', inplace=True)
        X.GarageFinish.fillna('Unf', inplace=True)
        X.GarageQual.fillna('TA', inplace=True)
        X.GarageCond.fillna('TA', inplace=True)
        X.PoolQC.fillna("NoPool", inplace=True)
        return X

class RemoveColumns(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and removes irrelevant, repetative, or corrupt data
    Decisions based on EDA found in ML_Final_EDA.ipynb
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X.drop(
        ['GarageArea', 'BsmtHalfBath', 'MiscVal', 'MoSold',
        'YrSold', 'BsmtFinSF2', 'LowQualFinSF', '3SsnPorch', 
        'Utilities', 'Exterior2nd', 'Alley', 'Id'],
        axis = 1)

class AlterAttributes(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and alter some attributes 
    Decisions based on EDA found in ML_Final_EDA.ipynb
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        #Combine GarageCars == 3 and GarageCars == 4
        X.GarageCars = X.GarageCars.map(lambda x: 3 if (x == 4) else x)

        #Fill Nans in GarageYrBlt with median year
        X.GarageYrBlt = X.GarageYrBlt.fillna(X.GarageYrBlt.median())

        #Combine KitchenAbvGr 1 and 0 AND 2 and 3 such that it is binary
        X.KitchenAbvGr = X.KitchenAbvGr.map(lambda x: 0 if (x == 1) else x)
        X.KitchenAbvGr = X.KitchenAbvGr.map(lambda x: 1 if (x == 2 or x == 3) else x)

        #Make Pool Area binary (x>0 or x==0)--> 1 if they have pool else 0
        X.PoolArea = X.PoolArea.map(lambda x: 1 if (x>0) else x)
        return X

class GetDummies(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame describing houses and convert all categorical attributes to one-hot encoded columns
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.PriorData = pd.get_dummies(pd.DataFrame(X))
        self.columns = self.PriorData.columns
        return self
    def transform(self, X):
        temp = pd.get_dummies(X)
        for col in self.columns:
            if col not in temp.columns:
                temp_ar = np.zeros(temp.shape[0])
                for i in range(len(temp_ar)):
                    temp_ar[i]=self.PriorData[col].median()
                temp[col] = temp_ar
                
        for i,j in zip(temp.isnull().sum(), temp.columns):
            if i>0:
                temp[j].fillna(temp[j].median(), inplace=True)
        return pd.get_dummies(temp)

class ScaleData(BaseEstimator, TransformerMixin):
    """
    Get a DataFrame, center & scale it using the mean and standard devision 
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.PriorData = X
        return self
    def transform(self, X):
        return (X-self.PriorData.mean())/self.PriorData.std()    

class GetDataFrame(BaseEstimator, TransformerMixin):
    """
    Get any data object and convert it to a Pandas DataFrame
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        self.columns = pd.DataFrame(X).columns
        return self
    def transform(self, X):
        return pd.DataFrame(X, columns=self.columns).fillna(0)

class VectorQuantization(BaseEstimator, TransformerMixin):
    """
    Creates cluster models of groups of related numerical attributes
    Adds these clusters as categorical attributes to the train and test data in tranform
    Grouping, reasoning, and descriptions of clusters shown in ML_Final_VQ.ipynb
    """
    def __init__(self): pass
    def fit(self, X, y=None):
        
        self.basement_cm = KMeans(n_clusters=3).fit(X[basement])
        self.outdoors_cm = KMeans(n_clusters=5).fit(X[outdoors])
        self.rooms_cm = KMeans(n_clusters=3).fit(X[rooms])
        self.floors_cm = KMeans(n_clusters=4).fit(X[floors])
        self.therest_cm = KMeans(n_clusters=2).fit(X[therest])
        self.cols = ['b1', 'b2', 'b3', 'o1', 'o2', 'o3', 'o4', 'o5', 'r1', 'r2', 'r3', 
               'f1', 'f2', 'f3', 'f4', 't1', 't2']
        
        return self
    def transform(self, X):

        temp_b = pd.get_dummies(self.basement_cm.predict(X[basement]))
        temp_b.columns = temp_b.columns.map(lambda x: "b"+str(x+1))
        temp_b.index = X.index

        temp_o = pd.get_dummies(self.outdoors_cm.predict(X[outdoors]))
        temp_o.columns = temp_o.columns.map(lambda x: "o"+str(x+1))
        temp_o.index = X.index
        
        temp_r = pd.get_dummies(self.rooms_cm.predict(X[rooms]))
        temp_r.columns = temp_r.columns.map(lambda x: "r"+str(x+1))
        temp_r.index = X.index
        
        temp_f = pd.get_dummies(self.floors_cm.predict(X[floors]))
        temp_f.columns = temp_f.columns.map(lambda x: "f"+str(x+1))
        temp_f.index = X.index
        
        temp_t = pd.get_dummies(self.therest_cm.predict(X[therest]))
        temp_t.columns = temp_t.columns.map(lambda x: "t"+str(x+1))
        temp_t.index = X.index
        
        #return pd.concat([temp_b, temp_o, temp_r, temp_f, temp_t, X], axis=1)
        ret = pd.concat([temp_b,temp_o, temp_r, temp_f, temp_t,  X], axis=1)
        for col in self.cols:
            if col not in ret.columns:
                print col
                ret[col] = np.zeros(len(ret))
        return ret
        

Prep = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization())
])

In [5]:
# Test pipeline fit
print "Number of Nans:",Prep.fit_transform(df.drop(['SalePrice'], axis=1)[0:1000]).isnull().sum().sum()
Prep.fit_transform(df.drop(['SalePrice'], axis=1)[0:1000]).head()

Number of Nans: 0


,b1,b2,b3,o1,o2,o3,o4,o5,r1,r2,r3,f1,f2,f3,f4,t1,t2,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBasement,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NoBasement,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBasement,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBasement,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBasement,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFP,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_NoPool,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFence,MiscFeature_Gar2,MiscFeature_NoShed,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleC

In [6]:
# Test pipeline transform
print "Number of Nans:",Prep.transform(df.drop(['SalePrice'], axis=1)[1000:]).isnull().sum().sum()
Prep.transform(df.drop(['SalePrice'], axis=1)[1000:]).head()

Number of Nans: 0


,b1,b2,b3,o1,o2,o3,o4,o5,r1,r2,r3,f1,f2,f3,f4,t1,t2,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,PoolArea,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NoBasement,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NoBasement,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_NoBasement,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NoBasement,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NoBasement,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NoFP,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,PoolQC_Ex,PoolQC_Fa,PoolQC_NoPool,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NoFence,MiscFeature_Gar2,MiscFeature_NoShed,MiscFeature_Othr,MiscFeature_Shed,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleC

In [10]:
# Test new pipeline with VQ using GridSearch
GradB = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization()),
    #('prep', Prep),
    ('bag', GradientBoostingRegressor(loss = "huber", n_estimators=10000, 
                                      learning_rate=0.01, subsample = 0.7))
])

param_grid = [
    {
    'bag__n_estimators' : [5000, 15000],
    'bag__learning_rate' : [0.005, 0.01]
    }
]

grid_search2 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search2.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search2.cv_results_["mean_test_score"],grid_search2.cv_results_["params"])

[(0.015508957574300937,
  {'bag__learning_rate': 0.005, 'bag__n_estimators': 5000}),
 (0.015314397324750441,
  {'bag__learning_rate': 0.005, 'bag__n_estimators': 15000}),
 (0.015185721327898272,
  {'bag__learning_rate': 0.01, 'bag__n_estimators': 5000}),
 (0.015908829112004856,
  {'bag__learning_rate': 0.01, 'bag__n_estimators': 15000})]

In [12]:
param_grid = [
    {
    'bag__n_estimators' : [3000, 5000, 8000],
    'bag__subsample' : [0.5, 0.7]
    }
]

grid_search3 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search3.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search3.cv_results_["mean_test_score"],grid_search3.cv_results_["params"])

[(0.015004922601778773, {'bag__n_estimators': 3000, 'bag__subsample': 0.5}),
 (0.015481031870297698, {'bag__n_estimators': 3000, 'bag__subsample': 0.7}),
 (0.014808123055112418, {'bag__n_estimators': 5000, 'bag__subsample': 0.5}),
 (0.015158121147832514, {'bag__n_estimators': 5000, 'bag__subsample': 0.7}),
 (0.014842026574314115, {'bag__n_estimators': 8000, 'bag__subsample': 0.5}),
 (0.015445529997098369, {'bag__n_estimators': 8000, 'bag__subsample': 0.7})]

In [13]:
best_prior = Pipeline([
    ('prep', Prep), 
    ('bag', GradientBoostingRegressor(n_estimators=30000, loss='huber', subsample=0.4, learning_rate=0.003))
])

res = cross_val_score(best_prior, df.drop("SalePrice", axis=1), df.SalePrice, cv=3, scoring=make_scorer(mean_squared_log_error))
res, res.mean()

(array([ 0.01227163,  0.01775844,  0.01441157]), 0.014813881658951131)

In [14]:
param_grid = [
    {
    'bag__n_estimators' : [4000, 5000, 6000],
    'bag__subsample' : [0.4, 0.5, 0.6]
    }
]

grid_search3 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search3.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search3.cv_results_["mean_test_score"],grid_search3.cv_results_["params"])

[(0.015237653281931541, {'bag__n_estimators': 4000, 'bag__subsample': 0.4}),
 (0.015295136156809271, {'bag__n_estimators': 4000, 'bag__subsample': 0.5}),
 (0.015081905360496504, {'bag__n_estimators': 4000, 'bag__subsample': 0.6}),
 (0.015098785682537665, {'bag__n_estimators': 5000, 'bag__subsample': 0.4}),
 (0.015019082846065073, {'bag__n_estimators': 5000, 'bag__subsample': 0.5}),
 (0.015296365104728786, {'bag__n_estimators': 5000, 'bag__subsample': 0.6}),
 (0.015202639952146827, {'bag__n_estimators': 6000, 'bag__subsample': 0.4}),
 (0.014821759635148779, {'bag__n_estimators': 6000, 'bag__subsample': 0.5}),
 (0.015143890395072818, {'bag__n_estimators': 6000, 'bag__subsample': 0.6})]

In [16]:
# Test new pipeline with VQ using GridSearch
GradB = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization()),
    ('bag', GradientBoostingRegressor(loss = "huber", subsample = 0.4))
])

param_grid = [
    {
    'bag__n_estimators' : [30000, 40000],
    'bag__learning_rate' : [0.005]
    }
]

grid_search2 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search2.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search2.cv_results_["mean_test_score"],grid_search2.cv_results_["params"])

[(0.015298638082986163,
  {'bag__learning_rate': 0.005, 'bag__n_estimators': 30000}),
 (0.01513798969914, {'bag__learning_rate': 0.005, 'bag__n_estimators': 40000})]

In [17]:
# Test new pipeline with VQ using GridSearch
GradB = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization()),
    ('bag', GradientBoostingRegressor(loss = "huber", subsample = 0.4))
])

param_grid = [
    {
    'bag__n_estimators' : [6000, 5000],
    'bag__learning_rate' : [0.01]
    }
]

grid_search2 = GridSearchCV(GradB, param_grid, cv=3, scoring=make_scorer(mean_squared_log_error))
grid_search2.fit(df.drop(["SalePrice"], axis=1), df.SalePrice)
zip(grid_search2.cv_results_["mean_test_score"],grid_search2.cv_results_["params"])

[(0.014927500349581537,
  {'bag__learning_rate': 0.01, 'bag__n_estimators': 6000}),
 (0.014875054467291154,
  {'bag__learning_rate': 0.01, 'bag__n_estimators': 5000})]

# Try it on the test data and make a submission to Kaggle

In [23]:
# Train on entire training dataset 
GradB = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization()),
    ('bag', GradientBoostingRegressor(loss = "huber", n_estimators=5000, 
                                      learning_rate=0.01, subsample = 0.5))
])

GradB.fit(df.drop(['SalePrice'],axis=1), df.SalePrice)

df_test = pd.read_csv("test.csv")
test_predictions = GradB.predict(df_test)

pd.DataFrame(test_predictions).describe()

,0
count,1459.000000
mean,178994.369959
std,76870.257315
min,29797.338100
25%,128703.067280
50%,159361.828896
75%,209153.641442
max,552710.570050


In [24]:
df_test_predictions = pd.DataFrame([df_test.Id.astype(int) , test_predictions]).T
df_test_predictions.columns = ['Id', 'SalePrice']
df_test_predictions['Id'] = df_test_predictions['Id'].astype(int)
df_test_predictions.to_csv("Submission_4.csv", index=False)

In [25]:
df_test_predictions

,Id,SalePrice
0,1461,124775.032907
1,1462,160998.441022
2,1463,191384.275497
3,1464,195265.891045
4,1465,185887.363412
5,1466,176827.108412
6,1467,170561.388105
7,1468,168301.015634
8,1469,186598.901114
9,1470,131437.300694


In [26]:
# Train on entire training dataset 
GradB = Pipeline([
    ('fillnanvalues', FillNanValues()),
    ('removecolumns', RemoveColumns()),
    ('alterattributes', AlterAttributes()),
    ('getdummies', GetDummies()),
    ('scaledata', ScaleData()),
    ('getdataframe', GetDataFrame()),
    ('vq', VectorQuantization()),
    ('bag', GradientBoostingRegressor(loss = "huber", n_estimators=15000, 
                                      learning_rate=0.005, subsample = 0.5))
])

GradB.fit(df.drop(['SalePrice'],axis=1), df.SalePrice)

df_test = pd.read_csv("test.csv")
test_predictions = GradB.predict(df_test)

pd.DataFrame(test_predictions).describe()
df_test_predictions = pd.DataFrame([df_test.Id.astype(int) , test_predictions]).T
df_test_predictions.columns = ['Id', 'SalePrice']
df_test_predictions['Id'] = df_test_predictions['Id'].astype(int)
df_test_predictions.to_csv("Submission_4.csv", index=False)
df_test_predictions

,Id,SalePrice
0,1461,126345.894856
1,1462,162078.950351
2,1463,192516.416455
3,1464,194003.475232
4,1465,181182.524853
5,1466,175102.195826
6,1467,170923.069909
7,1468,168364.628547
8,1469,183888.423042
9,1470,130344.707008
